# setPath

In [1]:
from sys import path
path.append(r"../")

# Twins-0-16-8

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.VSDD import run as run_VSDD

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2022,type=int,help='The random seed')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='the coefs between VZCU when computing t')
    argparser.add_argument('--VX',default=0,type=int,help='nonlinear relation between v and x')
    argparser.add_argument('--mV',default=0,type=int,help='The dim of Outside Instrumental variables V')
    argparser.add_argument('--mX',default=16,type=int,help='The dim of Observerd variables X')
    argparser.add_argument('--mU',default=8,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # Syn
    argparser.add_argument('--syn_alpha',default=0.1,type=float,help='The ratio of treament prediction loss')
    argparser.add_argument('--syn_beta',default=0.1,type=float,help='The ratio of disentangle loss')
    argparser.add_argument('--syn_gamma',default=0.01,type=float,help='The ratio of imb loss')
    argparser.add_argument('--syn_lambda',default=0.0001,type=float,help='The ratio of regularization loss')
    argparser.add_argument('--syn_twoStage',default=True,type=bool,help='whether use twostage method')
    argparser.add_argument('--lrate',default=5e-4,type=float,help='lrate')
    argparser.add_argument('--iteration',default=1000,type=float,help='iteration')
    argparser.add_argument('--output_delay',default=100,type=float,help='iteration')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Twins_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log_VSDD.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
alpha = args.syn_alpha
beta = args.syn_beta
gamma = args.syn_gamma
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/train.csv')
    val_df = pd.read_csv(dataDir + f'{exp}/val.csv')
    test_df = pd.read_csv(dataDir + f'{exp}/test.csv')

    train = CausalDataset(train_df, variables = ['u','x','v','z','a','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','a','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','a','p','s','m','t','g','y','f','c'])

    res_list = []

   
    obj_val, final = run_VSDD(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    res = np.array(res_list) 
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ alpha+data_cls for alpha in ['VSDD'] for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'VSDD_result.csv', index=False)
res_df

## Twins-4-16-8

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.VSDD_IV import run as run_VSDD

os.environ["CUDA_VISIBLE_DEVICES"] = '3'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2022,type=int,help='The random seed')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='the coefs between VZCU when computing t')
    argparser.add_argument('--VX',default=0,type=int,help='nonlinear relation between v and x')
    argparser.add_argument('--mV',default=4,type=int,help='The dim of Outside Instrumental variables V')
    argparser.add_argument('--mX',default=16,type=int,help='The dim of Observerd variables X')
    argparser.add_argument('--mU',default=8,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # Syn
    argparser.add_argument('--syn_alpha',default=0.1,type=float,help='The ratio of treament prediction loss')
    argparser.add_argument('--syn_beta',default=0.1,type=float,help='The ratio of disentangle loss')
    argparser.add_argument('--syn_gamma',default=0.01,type=float,help='The ratio of imb loss')
    argparser.add_argument('--syn_lambda',default=0.0001,type=float,help='The ratio of regularization loss')
    argparser.add_argument('--syn_twoStage',default=True,type=bool,help='whether use twostage method')
    argparser.add_argument('--lrate',default=5e-4,type=float,help='lrate')
    argparser.add_argument('--iteration',default=1000,type=float,help='iteration')
    argparser.add_argument('--output_delay',default=100,type=float,help='iteration')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Twins_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log_VSDD.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
alpha = args.syn_alpha
beta = args.syn_beta
gamma = args.syn_gamma
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/train.csv')
    val_df = pd.read_csv(dataDir + f'{exp}/val.csv')
    test_df = pd.read_csv(dataDir + f'{exp}/test.csv')

    train = CausalDataset(train_df, variables = ['u','x','v','z','a','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','a','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','a','p','s','m','t','g','y','f','c'])

    res_list = []

   
    obj_val, final = run_VSDD(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    res = np.array(res_list) 
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ alpha+data_cls for alpha in ['VSDD'] for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'VSDD_result.csv', index=False)
res_df

# Twins-0-16-12

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.VSDD import run as run_VSDD

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2022,type=int,help='The random seed')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='the coefs between VZCU when computing t')
    argparser.add_argument('--VX',default=0,type=int,help='nonlinear relation between v and x')
    argparser.add_argument('--mV',default=0,type=int,help='The dim of Outside Instrumental variables V')
    argparser.add_argument('--mX',default=16,type=int,help='The dim of Observerd variables X')
    argparser.add_argument('--mU',default=12,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # Syn
    argparser.add_argument('--syn_alpha',default=0.1,type=float,help='The ratio of treament prediction loss')
    argparser.add_argument('--syn_beta',default=0.1,type=float,help='The ratio of disentangle loss')
    argparser.add_argument('--syn_gamma',default=0.01,type=float,help='The ratio of imb loss')
    argparser.add_argument('--syn_lambda',default=0.0001,type=float,help='The ratio of regularization loss')
    argparser.add_argument('--syn_twoStage',default=True,type=bool,help='whether use twostage method')
    argparser.add_argument('--lrate',default=5e-4,type=float,help='lrate')
    argparser.add_argument('--iteration',default=2000,type=float,help='iteration')
    argparser.add_argument('--output_delay',default=100,type=float,help='iteration')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Twins_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log_VSDD.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
alpha = args.syn_alpha
beta = args.syn_beta
gamma = args.syn_gamma
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/train.csv')
    val_df = pd.read_csv(dataDir + f'{exp}/val.csv')
    test_df = pd.read_csv(dataDir + f'{exp}/test.csv')

    train = CausalDataset(train_df, variables = ['u','x','v','z','a','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','a','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','a','p','s','m','t','g','y','f','c'])

    res_list = []

   
    obj_val, final = run_VSDD(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    res = np.array(res_list) 
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ alpha+data_cls for alpha in ['VSDD'] for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'VSDD_result.csv', index=False)
res_df

# Twins-0-20-8

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.VSDD import run as run_VSDD

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2022,type=int,help='The random seed')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='the coefs between VZCU when computing t')
    argparser.add_argument('--VX',default=0,type=int,help='nonlinear relation between v and x')
    argparser.add_argument('--mV',default=0,type=int,help='The dim of Outside Instrumental variables V')
    argparser.add_argument('--mX',default=20,type=int,help='The dim of Observerd variables X')
    argparser.add_argument('--mU',default=8,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # Syn
    argparser.add_argument('--syn_alpha',default=1,type=float,help='The ratio of treament prediction loss')
    argparser.add_argument('--syn_beta',default=1,type=float,help='The ratio of disentangle loss')
    argparser.add_argument('--syn_gamma',default=0.001,type=float,help='The ratio of imb loss')
    argparser.add_argument('--syn_lambda',default=0.0001,type=float,help='The ratio of regularization loss')
    argparser.add_argument('--syn_twoStage',default=True,type=bool,help='whether use twostage method')
    argparser.add_argument('--lrate',default=5e-4,type=float,help='lrate')
    argparser.add_argument('--iteration',default=300,type=float,help='iteration')
    argparser.add_argument('--output_delay',default=5,type=float,help='iteration')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")
    
# set path
which_benchmark = 'Twins_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log_VSDD.txt'

if args.rewrite_log:
    f = open(logfile,'w')
    f.close()

results = []
alpha = args.syn_alpha
beta = args.syn_beta
gamma = args.syn_gamma
for exp in range(args.num_reps):
    # load data
    train_df = pd.read_csv(dataDir + f'{exp}/train.csv')
    val_df = pd.read_csv(dataDir + f'{exp}/val.csv')
    test_df = pd.read_csv(dataDir + f'{exp}/test.csv')

    train = CausalDataset(train_df, variables = ['u','x','v','z','a','p','s','m','t','g','y','f','c'])
    val = CausalDataset(val_df, variables = ['u','x','v','z','a','p','s','m','t','g','y','f','c'])
    test = CausalDataset(test_df, variables = ['u','x','v','z','a','p','s','m','t','g','y','f','c'])

    res_list = []

   
    obj_val, final = run_VSDD(exp, args, dataDir, resultDir, train, val, test, device)
    res_list = res_list + [obj_val['ate_train'],obj_val['ate_test']]
    
    res = np.array(res_list) 
    results.append(res)

results.append(np.mean(results,0))
results.append(np.std(results,0))
    
res_df = pd.DataFrame(np.array(results),
                       columns=[ alpha+data_cls for alpha in ['VSDD'] for data_cls in ['_train', '_test']]).round(4)
res_df.to_csv(resultDir + f'VSDD_result.csv', index=False)
res_df